In [1]:
%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
from operator import itemgetter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [4]:
OPENAI_API_KEY = ""

In [5]:
vectorstore = FAISS.from_texts(["praveen worked at Hyderabad"],embedding =OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))
retriever = vectorstore.as_retriever()

In [18]:
template = """
Answer the question based on the following context:
{context}

Question:{question}
"""

In [19]:
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question based on the following context:\n{context}\n\nQuestion:{question}\n'))])

In [20]:
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [21]:
chain = (
    {"context":retriever,"question":RunnablePassthrough()}
    |
    prompt
    |
    model
    |
    StrOutputParser()

)

In [22]:
chain.invoke("where did Praveen work? ")

'Praveen worked at Hyderabad.'

In [24]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)
chain.invoke({"question": "where did harrison work", "language": "telugu"})


'హ్యారిసన్ హైదరాబాద్\u200cలో పని చేశాడు.'